# Import Dependencies

In [66]:
# Basics
import pandas as pd #DataFrame
import numpy as np #Linear Algebra

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

#Model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from xgboost import XGBRegressor

#metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

#pipeline
from sklearn.pipeline import Pipeline

#os
import os

# Load Data

In [39]:
train = pd.read_csv('new_data/train.csv')
test = pd.read_csv('new_data/test.csv')

In [40]:
train.head()

,index,contest-tmp2m-14d__tmp2m,nmme0-tmp2m-34w__nmme0mean,nmme0-tmp2m-34w__nasa0,wind-uwnd-250-2010-1,wind-uwnd-925-2010-1,nasa0,nmme0mean,nmme-tmp2m-34w__nmmemean,nmme-tmp2m-34w__nasa,nmme-tmp2m-34w__gfdlflorb,nmme-tmp2m-34w__gfdlflora,nmme-tmp2m-56w__gfdlflorb,nmme-tmp2m-34w__gfdl,nmme-tmp2m-34w__cfsv2,contest-wind-h100-14d__wind-hgt-100,nmme-tmp2m-34w__ccsm4,nmme-tmp2m-34w__cancm4,nmme-tmp2m-34w__cancm3,cancm30,cfsv20,nmme-tmp2m-56w__nasa,nmme-tmp2m-56w__nmmemean,nmme-tmp2m-56w__gfdlflora,nmme0-tmp2m-34w__gfdl0,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme-tmp2m-56w__gfdl,nmme0-tmp2m-34w__gfdlflora0,nmme0-tmp2m-34w__gfdlflorb0,nmme0-tmp2m-34w__cfsv20,nmme-tmp2m-56w__cancm3,nmme-tmp2m-56w__cancm4,nmme-tmp2m-56w__ccsm3,nmme-tmp2m-56w__ccsm4,nmme-tmp2m-56w__cfsv2,sst-2010-1,nmme-tmp2m-34w__ccsm3,gfdlflorb0,contest-pevpr-sfc-gauss-14d__pevpr,cancm40,gfdl0,ccsm30,ccsm40,contest-wind-h500-14d__wind-hgt-500,gfdlflora0,contest-wind-h10-14d__wind-hgt-10,contest-prwtr-eatm-14d__prwtr,icec-2010-9,wind-hgt-850-2010-7,icec-2010-2,contest-wind-vwnd-250-14d__wind-vwnd-250,contest-wind-h850-14d__wind-hgt-850,wind-uwnd-250-2010-18,wind-hgt-500-2010-9,sst-2010-3,wind-vwnd-250-2010-7,wind-hgt-10-2010-4,wind-hgt-10-2010-5
0,0,28.744480,30.46,29.66,628.66,143.64,29.51,30.65,28.17,28.55,28.3,28.42,28.72,28.27,27.3,16666.81,27.98,29.34,27.83,30.18,29.33,28.38,28.11,27.7,30.56,29.02,31.64,29.57,30.73,28.25,31.52,31.68,29.71,28.3,29.47,27.13,27.36,27.71,352.20,27.57,31.45,237.00,32.86,31.33,28.85,30.73,5899.66,31.66,31246.63,42.45,0.03,77.17,0.97,-3.52,1535.52,35.85,1549.20,-19.69,-125.92,-4560.34,7128.13
1,1,28.370585,30.46,29.66,615.58,143.41,29.51,30.65,28.17,28.55,28.3,28.42,28.72,28.27,27.3,16667.31,27.98,29.34,27.83,30.18,29.33,28.38,28.11,27.7,30.56,29.02,31.64,29.57,30.73,28.25,31.52,31.68,29.71,28.3,29.47,27.13,27.36,27.71,350.96,27.57,31.45,228.90,32.86,31.33,28.85,30.73,5901.03,31.66,31244.78,42.66,0.03,77.48,0.97,-4.49,1538.00,28.76,1667.04,-20.66,-109.57,-5318.37,8109.37
2,2,28.133059,30.46,29.66,602.14,145.35,29.51,30.65,28.17,28.55,28.3,28.42,28.72,28.27,27.3,16668.39,27.98,29.34,27.83,30.18,29.33,28.38,28.11,27.7,30.56,29.02,31.64,29.57,30.73,28.25,31.52,31.68,29.71,28.3,29.47,27.13,27.36,27.71,349.86,27.57,31.45,220.69,32.86,31.33,28.85,30.73,5902.18,31.66,31239.27,43.23,0.03,116.42,0.97,-5.44,1540.32,24.45,1743.11,-21.34,-88.66,-5947.60,9137.52
3,3,28.256798,30.46,29.66,589.63,146.90,29.51,30.65,28.17,28.55,28.3,28.42,28.72,28.27,27.3,16667.39,27.98,29.34,27.83,30.18,29.33,28.38,28.11,27.7,30.56,29.02,31.64,29.57,30.73,28.25,31.52,31.68,29.71,28.3,29.47,27.13,27.36,27.71,348.91,27.57,31.45,225.28,32.86,31.33,28.85,30.73,5903.07,31.66,31232.86,43.11,0.03,183.76,0.97,-5.76,1541.10,20.95,1746.57,-21.92,-77.34,-6369.78,10010.87
4,4,28.372353,30.46,29.66,576.23,148.67,29.51,30.65,28.17,28.55,28.3,28.42,28.72,28.27,27.3,16665.65,27.98,29.34,27.83,30.18,29.33,28.38,28.11,27.7,30.56,29.02,31.64,29.57,30.73,28.25,31.52,31.68,29.71,28.3,29.47,27.13,27.36,27.71,348.03,27.57,31.45,237.24,32.86,31.33,28.85,30.73,5903.36,31.66,31226.16,42.98,0.03,264.03,0.97,-6.09,1539.73,11.62,1698.71,-22.54,-78.18,-6673.71,10458.26


In [41]:
test.head()

,index,nmme0-tmp2m-34w__nmme0mean,nmme0-tmp2m-34w__nasa0,wind-uwnd-250-2010-1,wind-uwnd-925-2010-1,nasa0,nmme0mean,nmme-tmp2m-34w__nmmemean,nmme-tmp2m-34w__nasa,nmme-tmp2m-34w__gfdlflorb,nmme-tmp2m-34w__gfdlflora,nmme-tmp2m-56w__gfdlflorb,nmme-tmp2m-34w__gfdl,nmme-tmp2m-34w__cfsv2,contest-wind-h100-14d__wind-hgt-100,nmme-tmp2m-34w__ccsm4,nmme-tmp2m-34w__cancm4,nmme-tmp2m-34w__cancm3,cancm30,cfsv20,nmme-tmp2m-56w__nasa,nmme-tmp2m-56w__nmmemean,nmme-tmp2m-56w__gfdlflora,nmme0-tmp2m-34w__gfdl0,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme-tmp2m-56w__gfdl,nmme0-tmp2m-34w__gfdlflora0,nmme0-tmp2m-34w__gfdlflorb0,nmme0-tmp2m-34w__cfsv20,nmme-tmp2m-56w__cancm3,nmme-tmp2m-56w__cancm4,nmme-tmp2m-56w__ccsm3,nmme-tmp2m-56w__ccsm4,nmme-tmp2m-56w__cfsv2,sst-2010-1,nmme-tmp2m-34w__ccsm3,gfdlflorb0,contest-pevpr-sfc-gauss-14d__pevpr,cancm40,gfdl0,ccsm30,ccsm40,contest-wind-h500-14d__wind-hgt-500,gfdlflora0,contest-wind-h10-14d__wind-hgt-10,contest-prwtr-eatm-14d__prwtr,icec-2010-9,wind-hgt-850-2010-7,icec-2010-2,contest-wind-vwnd-250-14d__wind-vwnd-250,contest-wind-h850-14d__wind-hgt-850,wind-uwnd-250-2010-18,wind-hgt-500-2010-9,sst-2010-3,wind-vwnd-250-2010-7,wind-hgt-10-2010-4,wind-hgt-10-2010-5
0,375734,30.37,29.39,675.14,162.81,29.28,30.77,28.06,27.94,27.83,28.28,27.91,27.79,27.22,16700.13,28.41,29.2,28.49,29.44,29.43,27.96,27.81,26.82,31.01,30.88,30.92,29.17,31.02,27.73,30.93,30.54,29.47,28.04,29.58,26.57,28.13,27.56,345.33,27.37,31.84,339.88,33.11,31.49,29.06,31.58,5916.86,31.71,31233.02,40.29,0.03,-1064.16,0.97,-4.82,1549.82,142.32,-529.25,-19.05,-211.71,-5435.01,7408.07
1,375735,30.37,29.39,671.74,161.69,29.28,30.77,28.06,27.94,27.83,28.28,27.91,27.79,27.22,16697.68,28.41,29.2,28.49,29.44,29.43,27.96,27.81,26.82,31.01,30.88,30.92,29.17,31.02,27.73,30.93,30.54,29.47,28.04,29.58,26.57,28.13,27.56,344.50,27.37,31.84,334.63,33.11,31.49,29.06,31.58,5917.85,31.71,31230.63,40.15,0.03,-828.01,0.97,-4.34,1550.55,140.27,-438.92,-19.20,-216.37,-5446.56,6913.60
2,375736,30.37,29.39,666.01,159.32,29.28,30.77,28.06,27.94,27.83,28.28,27.91,27.79,27.22,16694.94,28.41,29.2,28.49,29.44,29.43,27.96,27.81,26.82,31.01,30.88,30.92,29.17,31.02,27.73,30.93,30.54,29.47,28.04,29.58,26.57,28.13,27.56,343.40,27.37,31.84,337.83,33.11,31.49,29.06,31.58,5918.56,31.71,31225.26,40.07,0.03,-601.71,0.97,-3.62,1550.16,137.79,-325.51,-19.35,-204.15,-5740.40,6445.81
3,375737,30.37,29.39,660.21,157.58,29.28,30.77,28.06,27.94,27.83,28.28,27.91,27.79,27.22,16692.18,28.41,29.2,28.49,29.44,29.43,27.96,27.81,26.82,31.01,30.88,30.92,29.17,31.02,27.73,30.93,30.54,29.47,28.04,29.58,26.57,28.13,27.56,341.91,27.37,31.84,345.81,33.11,31.49,29.06,31.58,5920.30,31.71,31220.87,40.00,0.03,-490.60,0.97,-2.80,1550.15,134.52,-274.45,-19.38,-189.01,-6151.83,5895.94
4,375738,30.37,29.39,654.84,154.62,29.28,30.77,28.06,27.94,27.83,28.28,27.91,27.79,27.22,16689.71,28.41,29.2,28.49,29.44,29.43,27.96,27.81,26.82,31.01,30.88,30.92,29.17,31.02,27.73,30.93,30.54,29.47,28.04,29.58,26.57,28.13,27.56,340.22,27.37,31.84,357.39,33.11,31.49,29.06,31.58,5923.03,31.71,31214.54,39.80,0.03,-396.34,0.97,-2.59,1551.19,126.65,-279.28,-19.27,-182.05,-6527.22,5383.52


# Split into Train and Validation Data

In [42]:
y = train['contest-tmp2m-14d__tmp2m']
X = train.drop(['contest-tmp2m-14d__tmp2m'],axis=1)

In [43]:
X.head()

,index,nmme0-tmp2m-34w__nmme0mean,nmme0-tmp2m-34w__nasa0,wind-uwnd-250-2010-1,wind-uwnd-925-2010-1,nasa0,nmme0mean,nmme-tmp2m-34w__nmmemean,nmme-tmp2m-34w__nasa,nmme-tmp2m-34w__gfdlflorb,nmme-tmp2m-34w__gfdlflora,nmme-tmp2m-56w__gfdlflorb,nmme-tmp2m-34w__gfdl,nmme-tmp2m-34w__cfsv2,contest-wind-h100-14d__wind-hgt-100,nmme-tmp2m-34w__ccsm4,nmme-tmp2m-34w__cancm4,nmme-tmp2m-34w__cancm3,cancm30,cfsv20,nmme-tmp2m-56w__nasa,nmme-tmp2m-56w__nmmemean,nmme-tmp2m-56w__gfdlflora,nmme0-tmp2m-34w__gfdl0,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme-tmp2m-56w__gfdl,nmme0-tmp2m-34w__gfdlflora0,nmme0-tmp2m-34w__gfdlflorb0,nmme0-tmp2m-34w__cfsv20,nmme-tmp2m-56w__cancm3,nmme-tmp2m-56w__cancm4,nmme-tmp2m-56w__ccsm3,nmme-tmp2m-56w__ccsm4,nmme-tmp2m-56w__cfsv2,sst-2010-1,nmme-tmp2m-34w__ccsm3,gfdlflorb0,contest-pevpr-sfc-gauss-14d__pevpr,cancm40,gfdl0,ccsm30,ccsm40,contest-wind-h500-14d__wind-hgt-500,gfdlflora0,contest-wind-h10-14d__wind-hgt-10,contest-prwtr-eatm-14d__prwtr,icec-2010-9,wind-hgt-850-2010-7,icec-2010-2,contest-wind-vwnd-250-14d__wind-vwnd-250,contest-wind-h850-14d__wind-hgt-850,wind-uwnd-250-2010-18,wind-hgt-500-2010-9,sst-2010-3,wind-vwnd-250-2010-7,wind-hgt-10-2010-4,wind-hgt-10-2010-5
0,0,30.46,29.66,628.66,143.64,29.51,30.65,28.17,28.55,28.3,28.42,28.72,28.27,27.3,16666.81,27.98,29.34,27.83,30.18,29.33,28.38,28.11,27.7,30.56,29.02,31.64,29.57,30.73,28.25,31.52,31.68,29.71,28.3,29.47,27.13,27.36,27.71,352.20,27.57,31.45,237.00,32.86,31.33,28.85,30.73,5899.66,31.66,31246.63,42.45,0.03,77.17,0.97,-3.52,1535.52,35.85,1549.20,-19.69,-125.92,-4560.34,7128.13
1,1,30.46,29.66,615.58,143.41,29.51,30.65,28.17,28.55,28.3,28.42,28.72,28.27,27.3,16667.31,27.98,29.34,27.83,30.18,29.33,28.38,28.11,27.7,30.56,29.02,31.64,29.57,30.73,28.25,31.52,31.68,29.71,28.3,29.47,27.13,27.36,27.71,350.96,27.57,31.45,228.90,32.86,31.33,28.85,30.73,5901.03,31.66,31244.78,42.66,0.03,77.48,0.97,-4.49,1538.00,28.76,1667.04,-20.66,-109.57,-5318.37,8109.37
2,2,30.46,29.66,602.14,145.35,29.51,30.65,28.17,28.55,28.3,28.42,28.72,28.27,27.3,16668.39,27.98,29.34,27.83,30.18,29.33,28.38,28.11,27.7,30.56,29.02,31.64,29.57,30.73,28.25,31.52,31.68,29.71,28.3,29.47,27.13,27.36,27.71,349.86,27.57,31.45,220.69,32.86,31.33,28.85,30.73,5902.18,31.66,31239.27,43.23,0.03,116.42,0.97,-5.44,1540.32,24.45,1743.11,-21.34,-88.66,-5947.60,9137.52
3,3,30.46,29.66,589.63,146.90,29.51,30.65,28.17,28.55,28.3,28.42,28.72,28.27,27.3,16667.39,27.98,29.34,27.83,30.18,29.33,28.38,28.11,27.7,30.56,29.02,31.64,29.57,30.73,28.25,31.52,31.68,29.71,28.3,29.47,27.13,27.36,27.71,348.91,27.57,31.45,225.28,32.86,31.33,28.85,30.73,5903.07,31.66,31232.86,43.11,0.03,183.76,0.97,-5.76,1541.10,20.95,1746.57,-21.92,-77.34,-6369.78,10010.87
4,4,30.46,29.66,576.23,148.67,29.51,30.65,28.17,28.55,28.3,28.42,28.72,28.27,27.3,16665.65,27.98,29.34,27.83,30.18,29.33,28.38,28.11,27.7,30.56,29.02,31.64,29.57,30.73,28.25,31.52,31.68,29.71,28.3,29.47,27.13,27.36,27.71,348.03,27.57,31.45,237.24,32.86,31.33,28.85,30.73,5903.36,31.66,31226.16,42.98,0.03,264.03,0.97,-6.09,1539.73,11.62,1698.71,-22.54,-78.18,-6673.71,10458.26


In [44]:
y.head()

0    28.744480
1    28.370585
2    28.133059
3    28.256798
4    28.372353
Name: contest-tmp2m-14d__tmp2m, dtype: float64

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=3)

# Pipeline

In [46]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', XGBRegressor())
])


# Baseline Model

In [47]:
pipe.fit(X_train, y_train)

y_hat = pipe.predict(X_test)

### Accuracy

In [48]:
prediction_df = pd.concat([pd.Series(y_hat), X_test['index'].reset_index(drop=True)],axis=1)
index = ['contest-tmp2m-14d__tmp2m', 'index']
prediction_df.columns = index
prediction_df

,contest-tmp2m-14d__tmp2m,index
0,19.443832,329308
1,13.300938,295719
2,3.329164,300581
3,4.362996,16207
4,13.106089,23915
...,...,...
75142,12.675848,129579
75143,14.128506,68410
75144,5.357001,224253
75145,25.906443,60229


In [59]:
acc = r2_score(y_test, y_hat)
print("Accuracy:", acc)

Accuracy: 0.9926440040830077


In [60]:
accMSE = mean_squared_error(y_test, y_hat)
print("Accuracy:", accMSE)

Accuracy: 0.7171846788863872


In [64]:
accMAE = mean_absolute_error(y_test, y_hat)
print("Accuracy:", accMAE)

Accuracy: 0.6565045211178339


# Submission

In [65]:
prediction = pipe.predict(test)

In [68]:
prediction_df = pd.concat([pd.Series(y_hat), X_test['index'].reset_index(drop=True)],axis=1)
index = ['contest-tmp2m-14d__tmp2m', 'index']
prediction_df.columns = index
prediction_df.head()

,contest-tmp2m-14d__tmp2m,index
0,19.443832,329308
1,13.300938,295719
2,3.329164,300581
3,4.362996,16207
4,13.106089,23915


In [87]:
files = os.listdir('predictions')
if files is not None:
    highest = 0
    for i in files:
        curr = i[10:-4]
        highest = int(curr) if int(curr)>highest else highest

    file_name = f'prediction{highest+1}.csv'
else:
    file_name = 'prediction1.csv'


In [88]:
file_name

'prediction1.csv'

In [89]:
prediction_df.to_csv(f"predictions/{file_name}")